In [ ]:
import pandas as pd
import csv
import re
from lxml import etree

In [ ]:
#df_gnd = pd.read_csv('data/pataky-gnd-gefunden-03-03-24.csv')
df_cleaned_name = pd.read_csv('data/pataky/cleaned-namelist-pataky-all.csv')
df_biographie = pd.read_csv('data/pataky/pataky-biographie-bibliographie.csv')

In [ ]:
print(df_gnd.columns)
print(df_cleaned_name.columns)
print(df_biographie.columns)

In [ ]:
merged_gnd_cleaned = pd.merge(
    df_gnd,
    df_cleaned_name,
    left_on='name',
    right_on='Name Vorname',
    how='inner'  # Use 'inner' to keep only matching rows
)

In [ ]:
final_merged = pd.merge(
    merged_gnd_cleaned,
    df_biographie,
    left_on='Original',
    right_on='Name',
    how='inner' 
)

In [ ]:
final_merged.to_csv('test.csv')

In [ ]:
print(len(final_merged))
final_merged = final_merged.drop_duplicates()
print(len(final_merged))

In [ ]:
final_merged.to_csv('pataky-gnd-biblio-bio.csv', index=False)

In [ ]:
df_pataky = pd.read_csv('data/pataky/pataky-gnd-biblio-bio.csv')
print(len(df_pataky))
df_pataky = df_pataky.drop_duplicates()
print(len(df_pataky))

### STUFF

In [ ]:
df_pataky = pd.read_csv('data/pataky/pataky-gnd-biblio-bio.csv')

In [ ]:
df_pataky_flattened = pd.read_csv('data/pataky/pataky-flattened.csv')

In [ ]:
#### try regex aus biography Nach- und Vorname rausziehen -> klappt

# test1 = 'Nowotny, Isabella, Ps. Ida Klein, Herzen, Prag, Kleinseite'
# test2 = 'Bistram, Baronesse Ottilie von, Wiesbaden, Nicolasstr. 26, wurde am'
test3 = df_pataky_flattened['biography_1']
def use_regex(input_text):
    pattern = re.compile(r"(\w+(?:[\s-]\w+)*(?:\s+von\s+\w+)?),\s+((?:\w+\.?\s+)*\w+)", re.IGNORECASE)    
    match = pattern.match(input_text)
    if match:
        # input_text = input_text.replace(f"{match.group(1)}, {match.group(2)}", "")
        return f"{match.group(1)}, {match.group(2)}"
    return None
# mistakes = []
# for entry in test3:
#     res = use_regex(entry)
#     if res == None:
#         mistakes.append(entry)
# df_pataky_flattened['name'] = df_pataky_flattened['biography_1'].apply(use_regex)
# with open('mistakes.txt', 'w', encoding='utf-8') as f:
#     for line in mistakes:
#         f.write(f"{line}\n")

In [ ]:
for index, row in enumerate(df_pataky_flattened.iterrows()):
    tmp = df_pataky_flattened['name'].iloc[index]
    # Check if tmp is a string and not None/NaN
    if isinstance(tmp, str) and tmp in df_pataky_flattened['biography_1'].iloc[index]:
        df_pataky_flattened.at[index, 'biography_1'] = df_pataky_flattened['biography_1'].iloc[index].replace(tmp, "")

In [ ]:
df_pataky_flattened.to_csv('test.csv')

In [ ]:
sprachliste = pd.read_csv('data/pataky/sprachen.csv')
print(sprachliste['Sprache'])

In [ ]:
## explizite Sprachnennungen in Biographie finden

def use_regex_sprachen(input_text, sprachliste):
    return [phrase for phrase in sprachliste['Sprache'] if phrase.lower() in input_text.lower()]

explizite_sprachen = {}
for index, row in df_pataky.iterrows():
    tmp = use_regex_sprachen(row['biography'], sprachliste)
    if tmp:
        explizite_sprachen[row['gnd']] = tmp
print(explizite_sprachen)

In [ ]:
# (pd.DataFrame.from_dict(data=explizite_sprachen, orient='index')
#    .to_csv('data/pataky/pataky_explizite_sprachen_bio.csv', header=False))

In [ ]:
result = []

unique_ids = df_pataky['gnd'].unique()

for gnd_id in unique_ids:
    group = df_pataky[df_pataky['gnd'] == gnd_id].reset_index(drop=True)
    
    row_dict = {'gnd': gnd_id}
    
    for i, row in group.iterrows():
        suffix = i + 1
        for col in group.columns:
            if col != 'gnd': 
                row_dict[f"{col}_{suffix}"] = row[col]
    
    result.append(row_dict)

df_wide = pd.DataFrame(result)

In [ ]:
df_wide.to_csv('test.csv')

## WERKE

In [ ]:
###TEST

biblio = df_pataky['bibliography'][:10]
print(biblio)
#split_sample = sample.split('–')
for item in biblio:
    split_sample = [
        part.lstrip().replace("‒", "").strip() 
            for part in item.split('–')
        ]
print(split_sample)

In [ ]:
def parse_bibliography(entry):
    raw_entries = re.split(r'\s*‒\s*(?=[A-Z])', entry)
    # Clean the entries
    parsed_entries = [
        entry.strip() 
        for entry in raw_entries 
        if entry.strip()  
    ]
    
    return parsed_entries

In [ ]:
gnd_biblio = {}
for i in range(len(df_pataky)):
    temp = df_pataky['bibliography'].iloc[i]
    gnd = df_pataky['gnd'].iloc[i]
    
    if temp == '.':
        continue
    else:
        gnd_biblio[gnd] = parse_bibliography(temp)

In [ ]:
rows = []
for key, values in gnd_biblio.items():
    for value in values:
        rows.append((key, value))

with open('data/pataky/gnd-werke.csv', 'w', encoding='utf-8', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['GND', 'Werk'])  
    writer.writerows(rows) 

In [ ]:
df_werke = pd.read_csv('data/pataky/gnd-werke.csv')

In [ ]:
uebersetzungen = [{df_werke.loc[index, 'gnd']: werk} 
                  for index, werk in df_werke['werk'].items() 
                  if re.search(r'übers\w*', werk.lower())]

In [ ]:
with open("uebersetzungen.csv", mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Write the header
    writer.writerow(['gnd', 'werk'])

    # Write the data
    for item in uebersetzungen:
        for gnd, werk in item.items():
            writer.writerow([gnd, werk])


#### Pseudonyme und variante Namen matchen

In [ ]:
verweise_pseudonyme_list = ['Werke: s.', 'Biographie s.', 'Biographie und Werke s.', 'Ps.']

In [ ]:
def use_regex(input_text, phrases):
    phrases_pattern = '|'.join(re.escape(phrase) for phrase in phrases)
    pattern = re.compile(
        rf"({phrases_pattern})\s(?:([A-Za-z]+),\s([A-Za-z]+)|([A-Za-z]+)\s([A-Za-z]+))\.",
        re.IGNORECASE
    )
    match = pattern.search(input_text)
    print(match)
    if match:
        if match.group(2):
            return match.group(3) + " " + match.group(2)
        else:
            return match.group(4) + " " + match.group(5)
    return None

In [ ]:
test = "*Beaulieu, Gertraut Châles de, Ps. G. v. Beaulieu, Charlottenburg, Goethestrasse 7, entstammt einer französischen Emigrantenfamilie, Tochter eines Geheimen Oberjustizrats zu Berlin, geboren am 17. März 1846 zu Frankfurt a. O. Sie verbrachte lange Zeit auf Reisen und besuchte u.a. Italien, Griechenland, Spanien, Frankreich, Dänemark, Holland und Belgien, schriftstellerisch thätig war sie anfangs als Übersetzerin von Korrespondenzen ins Englische für den »Melbourne Argus« in Australien, die »Hour« in London. Schrieb für den »Capitan Fracassa«, Zeitung in Rom; dann redigierte sie 4 1/ 2 Iahr hindurch selbständig »Das humoristische Deutschland«, jetzt korrespondiert sie für das »Dagblad van Zuid-Holland en's Gravenhage«. Sie schrieb für »Westermanns Monatshefte« Reiseskizzen, ferner Skizzen, Novellen, Feuilletons für die »Nationalzeitung«, die »Vossische Zeitung«, »Zukunft«, »Über Land und Meer«, »Deutsche Revue«, »Illustrierte Zeitung« in Leipzig, »Kölnische Zeitung« u.a.m."
use_regex(test, verweise_pseudonyme_list)

In [ ]:
matched_verweise = {}
for index, row in df_pataky[130:140].iterrows():
    tmp = use_regex(row['biography'], verweise_pseudonyme_list)
    if tmp:
        matched_verweise[row['name']] = tmp

#### Orte zuordnen

In [ ]:
df_pataky = pd.read_csv('data/pataky/pataky_ohne_gnd.csv')

# list_orte = []
# with open("data/pataky/pataky_locations.txt", encoding="utf-8") as fp:  
#    for line in fp:
#        list_orte.extend(line.strip().split('\n'))
       
tmp = pd.read_csv('data/pataky/orte-geonames.csv')
list_orte = list(tmp['Ortsname'])       


In [ ]:
pataky_wohnorte = []

for index, text in enumerate(df_pataky['biography']):
    locations_per_author = []
    for ort in list_orte:
        if ort in text:
            locations_per_author.append(ort)
    pataky_wohnorte.append({df_pataky['identifier'].iloc[index] : locations_per_author})

In [ ]:
from collections import defaultdict
## [{x1 : [y1, y2, y3]}, {x1 : [z1, z2]},...] -> [{x1 : [[y1, y2, y3, z1, z2]}]

combined_dict = defaultdict(list)
for entry in pataky_wohnorte:
    for key, value in entry.items():
        combined_dict[key].extend(value)
        
pataky_wohnorte = [{key: values} for key, values in combined_dict.items()]

In [ ]:
with open("data/pataky/pataky_unidentified_wohnorte.csv", mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Write the header
    writer.writerow(['gnd', 'wohnort'])

    # Write the data
    for item in pataky_wohnorte:
        for gnd, wohnort in item.items():
            writer.writerow([gnd, wohnort])

In [ ]:
import requests

def api_request_geonames(place_name):
    url = "http://api.geonames.org/searchJSON"
    params = {
        'q': place_name,
        'maxRows': 10,
        'username': 'kitkatja'
    }
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        if 'geonames' in data:
            return [place['geonameId'] for place in data['geonames']]
        else:
            print("No results found")
            return []
    except requests.RequestException as e:
        print("Request failed:", e)
        return []
    except ValueError as e:
        print("Failed to parse JSON:", e)
        return []

In [ ]:
location_with_geonames = {}  

place_list = list_orte  
for place in place_list:
    result = api_request_geonames(place)  
    if result:  
        location_with_geonames[place] = result[0]  
    else:
        location_with_geonames[place] = 'NaN'  

print(location_with_geonames)

In [ ]:
with open('data/pataky/orte-geonames.csv', 'w', encoding='utf-8') as csv_file:  
    writer = csv.writer(csv_file)
    for key, value in location_with_geonames.items():
       writer.writerow([key, value])

### TEI mit placeName tags anreichern

In [ ]:
ortsliste_pataky = pd.read_csv('data/pataky/orte-geonames.csv')

In [ ]:
## iteriert durch die einzelnen Paragraphen des TEI und sucht in jedem nach den gespeicherten Orten in der Liste
## wenn Ort gefunden wird, wird <placeName /> drum geschrieben
## paragraph wird dann als etree XML object zurück ins TEI geschrieben
## Problem dabei: es muss in ein tag eingebettet sein, um als XML object erkannt zu werden -> doppelte <p> Auszeichnung im TEI -> nicht valide
## Lösung: nach Abschluss des Parsens die doppelten Tags manuell durch find+replace entfernen
try:
    tree = etree.parse('data/pataky/tei-xml/pataky_lexikon02.xml')
    root = tree.getroot()
    namespace = {"tei": "http://www.tei-c.org/ns/1.0"}
    text_elements = root.xpath("//tei:body//tei:p", namespaces=namespace)
    place_element = etree.Element("placeName")

    for paragraph in text_elements:
        full_text = " ".join(text.strip() for text in paragraph.itertext()).replace("\n", " ").strip()
        full_text = re.sub(r'\s+', ' ', full_text).strip()

        #print(f"Parsing paragraph: {full_text}")         
        for location in ortsliste_pataky['Ortsname']:
            if location in full_text:
                pattern = rf'\b{re.escape(location)}\b'
                full_text = re.sub(pattern, fr'<placeName xmlns="http://www.tei-c.org/ns/1.0">{location}</placeName>', full_text)
        paragraph.clear() 
        paragraph.append(etree.XML(f"<p>{full_text}</p>"))

    tree.write('data/pataky/tei-xml/pataky_lexikon01_modified-test.xml', pretty_print=True, encoding='UTF-8')            
except etree.XMLSyntaxError as e:
       print(f"XML parsing error:{full_text}, {e}")
except FileNotFoundError:
    print("File not found. Please check the file path.")

In [ ]:
### Autorinnen ohne gnd finden

df_ohnegnd = pd.read_csv('data/pataky/pataky-not-found-18-5-24.csv')
df_pataky = pd.read_csv('data/pataky/pataky-gnd-biblio-bio.csv')
result = df_ohnegnd[~df_ohnegnd['Name'].isin(df_pataky['name'])]

merged_gnd_cleaned = pd.merge(
    df_ohnegnd,
    df_cleaned_name,
    left_on='Name',
    right_on='Name Vorname',
    how='inner'  # Use 'inner' to keep only matching rows
)

final_merged = pd.merge(
    merged_gnd_cleaned,
    df_biographie,
    left_on='Original',
    right_on='Name',
    how='inner' 
)
final_merged.to_csv('pataky_ohne_gnd.csv', index=False)

In [24]:
#### Attribute zu persName hinzufügen

identifier_df = pd.read_csv('data/pataky/pataky-alle-identifier.csv')
identifier_df = identifier_df.drop_duplicates()

identifier_list = identifier_df.to_dict(orient="records")

name_id_map = {
    entry["name"]: f"id_{entry['identifier']}" for entry in identifier_list
}

tree = etree.parse("data/pataky/tei-xml/pataky_lexikon02_modified.xml")
root = tree.getroot()

ns = {'tei': 'http://www.tei-c.org/ns/1.0'}

for pers in root.xpath(".//tei:persName", namespaces=ns):
    name_text = pers.text.strip() if pers.text else ""
    if name_text in name_id_map:
        pers.attrib['{http://www.w3.org/XML/1998/namespace}id'] = name_id_map[name_text]

tree.write("data/pataky/tei-xml/pataky_lexikon02_attributes.xml", encoding="utf-8", xml_declaration=True, pretty_print=True)